<a href="https://colab.research.google.com/github/wndnjs2037/AIBasic_Boostcourse/blob/main/AI%EC%8A%A4%ED%84%B0%EB%94%94_5%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()

In [2]:
# MNIST Database -> 손글씨 숫자 이미지 집합
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 #트레이닝 15회 반복
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

In [3]:
# 데이터 로드하기
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

In [4]:
# 불러온 데이터를 학습할 모델 구현
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias = True).to(device)
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 0.6311, -0.2091,  0.8356,  ..., -1.3043,  0.3351,  0.9042],
        [-0.0761, -1.4224, -1.0800,  ..., -2.0003,  1.3290, -0.5558],
        [-0.0076, -0.7995, -2.0143,  ..., -0.4154,  0.1873, -0.4130],
        ...,
        [-0.3827,  0.4127, -0.4284,  ..., -0.1769,  1.4464, -0.6626],
        [-0.6733,  1.6255,  1.0745,  ...,  0.6309, -0.1514, -1.1572],
        [ 0.2494,  0.3885,  0.4520,  ...,  0.4121, -0.4813,  1.4511]],
       requires_grad=True)

In [5]:
# loss 함수, optimizer 구현하기 (가중치 업데이트용)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [6]:
# 학습 Loop 구현
for epoch in range(training_epochs):
  for i, (imgs, labels) in enumerate(train_loader):
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = imgs.view(-1, 28*28) #규격 설정

    outputs = linear(imgs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax).float().mean()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}]m Step [{}/{}], Loss: {:.4f}, Accuracy : {:2f}%'.format(
          epoch+1, training_epochs, i+1, len(test_loader), loss.item(), accuracy.item() *100))


Epoch [1/15]m Step [100/100], Loss: 3.0548, Accuracy : 47.999999%
Epoch [1/15]m Step [200/100], Loss: 1.8956, Accuracy : 66.000003%
Epoch [1/15]m Step [300/100], Loss: 2.0302, Accuracy : 63.999999%
Epoch [1/15]m Step [400/100], Loss: 1.6060, Accuracy : 69.000000%
Epoch [1/15]m Step [500/100], Loss: 1.3253, Accuracy : 72.000003%
Epoch [1/15]m Step [600/100], Loss: 1.0905, Accuracy : 75.999999%
Epoch [2/15]m Step [100/100], Loss: 0.9926, Accuracy : 76.999998%
Epoch [2/15]m Step [200/100], Loss: 1.5276, Accuracy : 73.000002%
Epoch [2/15]m Step [300/100], Loss: 0.6610, Accuracy : 83.999997%
Epoch [2/15]m Step [400/100], Loss: 0.7171, Accuracy : 85.000002%
Epoch [2/15]m Step [500/100], Loss: 1.1283, Accuracy : 75.000000%
Epoch [2/15]m Step [600/100], Loss: 1.3827, Accuracy : 73.000002%
Epoch [3/15]m Step [100/100], Loss: 0.9147, Accuracy : 81.999999%
Epoch [3/15]m Step [200/100], Loss: 0.9617, Accuracy : 75.999999%
Epoch [3/15]m Step [300/100], Loss: 0.9300, Accuracy : 79.000002%
Epoch [3/1

In [7]:
linear.eval()
with torch.no_grad():
  correct = 0
  total = 0

  for i, (imgs, labels) in enumerate(test_loader):
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = imgs.view(-1, 28 * 28)
    outputs = linear(imgs)
    _, argmax = torch.max(outputs, 1) # 최종 출력이 가장 높은 클래스 선택
    total += imgs.size(0)
    correct += (labels == argmax).sum().item()

  print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images: 89.26%
